In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import sys
oldsysstdout = sys.stdout
class flushfile():
    def __init__(self, f):
        self.f = f
    def __getattr__(self,name): 
        return object.__getattribute__(self.f, name)
    def write(self, x):
        self.f.write(x)
        self.f.flush()
    def flush(self):
        self.f.flush()
sys.stdout = flushfile(sys.stdout)

In [5]:
data = pd.read_csv('../12_fold_8_clf_train_probs.csv', index_col = 0)

In [7]:
import json

In [8]:
with open('../train.json', 'r') as f:
    train = json.load(f)
with open('../test.json', 'r') as f:
    test = json.load(f)

In [9]:
target = map(lambda x: x['cuisine'], train)

In [10]:
b, y = np.unique(target, return_inverse=True)

In [14]:
data.loc[0]

NB_0          8.691413e-14
NB_1          1.004268e-18
NB_2          5.267302e-11
NB_3          2.099679e-20
NB_4          1.092444e-14
NB_5          8.658133e-12
NB_6          9.998935e-01
NB_7          9.498148e-16
NB_8          1.224125e-18
NB_9          1.660152e-06
NB_10         3.296929e-16
NB_11         3.329920e-21
NB_12         3.078154e-20
NB_13         1.048374e-04
NB_14         1.021859e-11
NB_15         1.602825e-16
NB_16         8.762669e-14
NB_17         2.181474e-09
NB_18         1.161401e-18
NB_19         3.242252e-18
PAC_bag_0     0.000000e+00
PAC_bag_1     0.000000e+00
PAC_bag_2     0.000000e+00
PAC_bag_3     0.000000e+00
PAC_bag_4     0.000000e+00
PAC_bag_5     0.000000e+00
PAC_bag_6     1.000000e+00
PAC_bag_7     0.000000e+00
PAC_bag_8     0.000000e+00
PAC_bag_9     0.000000e+00
                  ...     
SVC_bag_10    0.000000e+00
SVC_bag_11    0.000000e+00
SVC_bag_12    0.000000e+00
SVC_bag_13    0.000000e+00
SVC_bag_14    0.000000e+00
SVC_bag_15    0.000000e+00
S

In [15]:
X = data.values.astype('float32')
y = y.astype('int32')

In [11]:
import lasagne
from lasagne.layers import InputLayer, DenseLayer, DropoutLayer
from lasagne.utils import floatX
import theano.tensor as T
import theano

Using gpu device 0: GeForce GTX 670 (CNMeM is disabled)


In [16]:
from sklearn.cross_validation import StratifiedKFold

In [17]:
cv = StratifiedKFold(y, n_folds=12, shuffle=True, random_state=0)

In [20]:
input_var = T.tensor3('inputs')
target_var = T.ivector('targets')

In [24]:
X.shape

(39774, 160)

In [25]:
def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(inputs.shape[0])
        np.random.shuffle(indices)
    for start_idx in range(0, inputs.shape[0] - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt].reshape((-1, 1, 160)), targets[excerpt]

In [28]:
import time
from copy import copy

In [30]:
from sklearn.metrics import accuracy_score

In [48]:
f = 1
accuracies = []
for train_index, test_index in cv:
    
    X_train = X[train_index]
    X_test = X[test_index]
    y_train = y[train_index]
    y_test = y[test_index]
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    y_train = y_train.astype('int32')
    y_test = y_test.astype('int32')
    
    net = {}
    net['input'] = InputLayer((None, 1, 160),input_var=input_var)
    net['fc1'] = DenseLayer(net['input'], num_units=1000,
                            nonlinearity = lasagne.nonlinearities.rectify,
                            W = lasagne.init.GlorotUniform(gain = 'relu'))
    net['drop1'] = DropoutLayer(net['fc1'], p=0.5)
    net['fc4'] = DenseLayer(net['drop1'], num_units=200,
                            nonlinearity = lasagne.nonlinearities.rectify,
                            W = lasagne.init.GlorotUniform(gain = 'relu'))
    net['drop4'] = DropoutLayer(net['fc4'], p=0.5)
    net['fc5'] = DenseLayer(net['drop4'], num_units=20, nonlinearity=lasagne.nonlinearities.softmax)
    output_layer = net['fc5']
    
    prediction = lasagne.layers.get_output(output_layer)
    loss = lasagne.objectives.categorical_crossentropy(prediction, target_var)
    loss = loss.mean()

    params = lasagne.layers.get_all_params(output_layer, trainable=True)
    updates = lasagne.updates.nesterov_momentum(
            loss, params, learning_rate=0.05, momentum=0.9)

    test_prediction = lasagne.layers.get_output(output_layer, deterministic=True)
    test_loss = lasagne.objectives.categorical_crossentropy(test_prediction,
                                                            target_var)
    test_loss = test_loss.mean()
    test_acc = T.mean(T.eq(T.argmax(test_prediction, axis=1), target_var),
                      dtype=theano.config.floatX)

    train_fn = theano.function([input_var, target_var], loss, updates=updates)

    val_fn = theano.function([input_var, target_var], [test_loss, test_acc])

    test_fn = theano.function([input_var], test_prediction)
    
    num_epochs = 40
    batchsize = 2500
    best_epoch = 0
    best_acc = 0
    best_out = np.zeros((len(X_test), 20))

    print("Starting training... fold {:} of {:}".format(f, 10))
    f += 1
    for epoch in range(num_epochs):
        train_err = 0
        train_batches = 0
        start_time = time.time()
        for batch in iterate_minibatches(X_train, y_train, batchsize, shuffle=True):
            inputs, targets = batch
            train_err += train_fn(inputs, targets)
            train_batches += 1
            
        print("Epoch {} of {} took {:.3f}s".format(epoch + 1, num_epochs, time.time() - start_time))
        print("  training loss:\t\t{:.6f}".format(train_err / train_batches))
        #print("  training accuracy:\t\t{:.6f}".format(train_acc / train_batches))
        #print("  validation loss:\t\t{:.6f}".format(val_err / val_batches))
        #print("  validation accuracy:\t\t{:.6f}".format(val_acc / val_batches))
    
    preds = []
    for t in X_test:
        vals = test_fn(t.reshape((1, 1, 160)))
        preds.append(vals)
        
    preds = np.array(preds).reshape((-1, 20)).argmax(axis = 1)
        
    print accuracy_score(y_test, preds)
    accuracies.append(accuracy_score(y_test, preds))
        
print np.mean(accuracies), np.std(accuracies)

Starting training... fold 1 of 10
Epoch 1 of 50 took 0.172s
  training loss:		2.196316
Epoch 2 of 50 took 0.165s
  training loss:		1.334671
Epoch 3 of 50 took 0.152s
  training loss:		1.065298
Epoch 4 of 50 took 0.148s
  training loss:		0.947153
Epoch 5 of 50 took 0.146s
  training loss:		0.883328
Epoch 6 of 50 took 0.147s
  training loss:		0.844075
Epoch 7 of 50 took 0.149s
  training loss:		0.821449
Epoch 8 of 50 took 0.166s
  training loss:		0.804069
Epoch 9 of 50 took 0.166s
  training loss:		0.793019
Epoch 10 of 50 took 0.153s
  training loss:		0.788725
Epoch 11 of 50 took 0.164s
  training loss:		0.777361
Epoch 12 of 50 took 0.154s
  training loss:		0.772614
Epoch 13 of 50 took 0.155s
  training loss:		0.765787
Epoch 14 of 50 took 0.164s
  training loss:		0.762143
Epoch 15 of 50 took 0.152s
  training loss:		0.755816
Epoch 16 of 50 took 0.159s
  training loss:		0.748885
Epoch 17 of 50 took 0.152s
  training loss:		0.748893
Epoch 18 of 50 took 0.151s
  training loss:		0.747677
Epo

In [49]:
import xgboost as xgb

In [54]:
params = {"objective": "multi:softmax", 
          "booster": "gbtree",
          "nthread": 4,
          "eta": 0.5, 
          "max_depth": 3, 
          "subsample": 0.9, 
          "colsample_bytree": 0.5,
          "num_class": 20,
          "silent": 1
         } 
num_trees = 50

In [55]:
f = 1
accuracies = []
for train_index, test_index in cv:
    
    X_train = X[train_index]
    X_test = X[test_index]
    y_train = y[train_index]
    y_test = y[test_index]
    dtrain = xgb.DMatrix(X_train, y_train)
    dvalid = xgb.DMatrix(X_test)
    #dtest = xgb.DMatrix(test[features])
    watchlist = [(dtrain, 'train')]
    gbm = xgb.train(params, dtrain, num_trees, evals=watchlist, verbose_eval=False)
    
    preds = gbm.predict(dvalid)
        
    print accuracy_score(y_test, preds)
    accuracies.append(accuracy_score(y_test, preds))
        
print np.mean(accuracies), np.std(accuracies)

0.805538832029
0.802227573751
0.806626506024
0.803496081977
0.804098854732
0.797044632087
0.80356065178


KeyboardInterrupt: 

In [58]:
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import cross_val_score

In [57]:
f = 1
accuracies = []
for train_index, test_index in cv:
    
    X_train = X[train_index]
    X_test = X[test_index]
    y_train = y[train_index]
    y_test = y[test_index]
    
    clf = LogisticRegression(C=4,penalty='l2',random_state=0,solver='lbfgs',multi_class='ovr')
    
    clf.fit(X_train, y_train)
    
    preds = clf.predict(X_test)
        
    print accuracy_score(y_test, preds)
    accuracies.append(accuracy_score(y_test, preds))
        
print np.mean(accuracies), np.std(accuracies)

0.823600240819
0.815171583384


KeyboardInterrupt: 

In [59]:
cross_val_score(LogisticRegression(C=4,penalty='l2',random_state=0,solver='lbfgs',multi_class='ovr'), X, y)

array([ 0.81565257,  0.81454107,  0.8148204 ])